In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.vector_ar.vecm import VECM
from statsmodels.stats.diagnostic import acorr_ljungbox

# Importación robusta de coint_johansen y kpss
try:
    from statsmodels.tsa.vector_ar.util import coint_johansen
except ImportError:
    try:
        from statsmodels.tsa.stattools import coint_johansen
    except ImportError:
        try:
            from statsmodels.tsa.vector_ar.vecm import coint_johansen
        except ImportError:
            print("Warning: coint_johansen not found. Will implement basic cointegration test.")
            coint_johansen = None

try:
    from statsmodels.tsa.stattools import kpss
except ImportError:
    print("Warning: kpss not available in this statsmodels version")
    kpss = None

import warnings
warnings.filterwarnings('ignore')

# Configuración de gráficos
plt.style.use('default')
try:
    sns.set_palette("husl")
except:
    pass

# DataFrame indicadores
df_indicadores = pd.read_csv('../data/indicadores/indicadores_macroeconomicos.csv',
                             sep=';', encoding='latin1')
df_indicadores = df_indicadores.set_index('fecha')
# CONVERTIR EL ÍNDICE A STRING
df_indicadores.index = df_indicadores.index.astype(str)
df_indicadores.index.name = 'fecha'

# DataFrame termotrade  
df_termotrade = pd.read_csv('../data/indicadores/monthly_results_brazil.csv', 
                            sep=";", encoding='latin1')
df_termotrade = df_termotrade[['date', 'ctot_level']]
df_termotrade = df_termotrade.rename(columns={'ctot_level': 'termoftrade'})
df_termotrade = df_termotrade.set_index('date')
df_termotrade.index = pd.to_datetime(df_termotrade.index).strftime('%Y%m')
df_termotrade.index.name = 'fecha'


# Hacer el merge de los dos DataFrames
df_combined = df_indicadores.join(df_termotrade, how='outer')


gdp_brazil = pd.read_csv('../data/QGDP/brazil.csv', sep =",", encoding='latin1')
gdp_brazil = gdp_brazil.set_index(gdp_brazil.columns[0])
gdp_brazil.index = pd.to_datetime(gdp_brazil.index).strftime('%Y%m')  # String YYYYMM

gdp_brazil = gdp_brazil.rename(columns={gdp_brazil.columns[0]: 'GDP_BRAZIL'})

c:\Users\Usuario\anaconda3\envs\tftimeseriesII\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Usuario\anaconda3\envs\tftimeseriesII\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\Usuario\anaconda3\envs\tftimeseriesII\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# PASO 1: CONSOLIDACIÓN DE DATOS (SIMPLIFICADO)
print("="*50)
print("🔍 PASO 1: CONSOLIDACIÓN DE DATOS")
print("="*50)

# Dataset final usando PIB como base (trimestral)
df_model = gdp_brazil.copy()
df_model = df_model.join(df_termotrade, how='inner')
df_model = df_model.join(df_indicadores, how='inner')

print(f"✅ Dataset consolidado: {df_model.shape}")
print(f"📅 Período: {df_model.index.min()}-{df_model.index.max()}")
print(f"🔢 Variables: {list(df_model.columns)}")
print("\n📊 Primeras observaciones:")
print(df_model.head())

🔍 PASO 1: CONSOLIDACIÓN DE DATOS
✅ Dataset consolidado: (99, 9)
📅 Período: 200004-202410
🔢 Variables: ['GDP_BRAZIL', 'termoftrade', 'PI_USA', 'PI_FRA', 'PI_GER', 'PI_ITA', 'PI_UK', 'DGS10', 'SPREAD_USA']

📊 Primeras observaciones:
        GDP_BRAZIL  termoftrade   PI_USA  PI_FRA  PI_GER  PI_ITA  PI_UK  \
200004    194948.4    99.921576  92.6659   110.5    84.8   121.3   77.1   
200007    197616.2    99.834027  92.8373   111.7    86.4   121.0   77.5   
200010    199860.6    99.660916  92.6400   112.2    86.7   122.4   78.7   
200101    200333.4    99.687145  91.8908   112.5    87.2   122.2   80.1   
200104    199596.3    99.700423  90.7384   111.9    86.0   120.1   80.7   

           DGS10  SPREAD_USA  
200004  5.990526    5.847381  
200007  6.054000    6.123333  
200010  5.738571    7.292273  
200101  5.160952    8.497826  
200104  5.141000    8.496667  


In [3]:
# PASO 2: VARIABLES DEL MODELO VECM (VERSIÓN CORREGIDA)
print("="*50)
print("📊 PASO 2: VARIABLES DEL MODELO (PONDERADO)")
print("="*50)

# Variables principales siguiendo metodología Talvi et al.
df_vecm = pd.DataFrame(index=df_model.index)

df_vecm['log_gdp_brazil'] = np.log(df_model['GDP_BRAZIL'])
df_vecm['log_tot_brazil'] = np.log(df_model['termoftrade'])

# IP_G7 con ponderación específica
df_vecm['log_ip_g7'] = np.log(df_model['PI_USA'] * 0.40 + 
                              df_model['PI_FRA'] * 0.15 + 
                              df_model['PI_GER'] * 0.20 + 
                              df_model['PI_ITA'] * 0.12 + 
                              df_model['PI_UK'] * 0.13)

df_vecm['us_10y'] = df_model['DGS10']
df_vecm['risk_spread'] = df_model['SPREAD_USA']

# Limpiar datos
df_vecm = df_vecm.dropna()

print(f"✅ Variables creadas (IP_G7 ponderado): {list(df_vecm.columns)}")
print(f"📊 Observaciones: {len(df_vecm)}")
print(f"📅 Período final: {df_vecm.index.min()}-{df_vecm.index.max()}")
print(f"\n📈 Estadísticas descriptivas:")
print(df_vecm.describe().round(3))

📊 PASO 2: VARIABLES DEL MODELO (PONDERADO)
✅ Variables creadas (IP_G7 ponderado): ['log_gdp_brazil', 'log_tot_brazil', 'log_ip_g7', 'us_10y', 'risk_spread']
📊 Observaciones: 99
📅 Período final: 200004-202410

📈 Estadísticas descriptivas:
       log_gdp_brazil  log_tot_brazil  log_ip_g7  us_10y  risk_spread
count          99.000          99.000     99.000  99.000       99.000
mean           12.504           4.615      4.593   3.269        5.464
std             0.161           0.009      0.048   1.290        2.609
min            12.180           4.601      4.327   0.624        2.740
25%            12.368           4.608      4.570   2.225        3.653
50%            12.581           4.613      4.600   3.388        4.663
75%            12.623           4.622      4.616   4.270        6.487
max            12.729           4.640      4.672   6.054       16.788


In [4]:
# PASO 3: TESTS DE RAÍCES UNITARIAS
print("="*50)
print("🔍 PASO 3: TESTS ECONOMÉTRICOS")
print("="*50)

def test_unit_root(series, name):
   """Test ADF para raíces unitarias"""
   adf_stat, adf_pval = adfuller(series.dropna())[:2]
   result = "No Estacionaria" if adf_pval > 0.05 else "Estacionaria"
   return {'variable': name, 'adf_stat': adf_stat, 'p_value': adf_pval, 'result': result}

# Tests en niveles
print("📊 TESTS DE RAÍCES UNITARIAS (NIVELES):")
unit_root_results = []
for col in df_vecm.columns:
   result = test_unit_root(df_vecm[col], col)
   unit_root_results.append(result)
   print(f"{col:<15}: p-val={result['p_value']:.3f} -> {result['result']}")

# Tests en primeras diferencias
print("\n📊 TESTS EN PRIMERAS DIFERENCIAS:")
diff_results = []
for col in df_vecm.columns:
   diff_series = df_vecm[col].diff().dropna()
   result = test_unit_root(diff_series, f"d_{col}")
   diff_results.append(result)
   print(f"d_{col:<14}: p-val={result['p_value']:.3f} -> {result['result']}")

🔍 PASO 3: TESTS ECONOMÉTRICOS
📊 TESTS DE RAÍCES UNITARIAS (NIVELES):
log_gdp_brazil : p-val=0.500 -> No Estacionaria
log_tot_brazil : p-val=0.221 -> No Estacionaria
log_ip_g7      : p-val=0.000 -> Estacionaria
us_10y         : p-val=0.167 -> No Estacionaria
risk_spread    : p-val=0.011 -> Estacionaria

📊 TESTS EN PRIMERAS DIFERENCIAS:
d_log_gdp_brazil: p-val=0.000 -> Estacionaria
d_log_tot_brazil: p-val=0.000 -> Estacionaria
d_log_ip_g7     : p-val=0.000 -> Estacionaria
d_us_10y        : p-val=0.000 -> Estacionaria
d_risk_spread   : p-val=0.000 -> Estacionaria


In [11]:
# PASO 4: PREPARACIÓN PARA VECM Y TEST DE COINTEGRACIÓN
print("="*50)
print("🔍 PASO 4: COINTEGRACIÓN")
print("="*50)



# Usar todas las 5 variables del modelo original
vecm_vars_full = ['log_gdp_brazil', 'log_ip_g7', 'log_tot_brazil', 'us_10y', 'risk_spread']
df_vecm_full = df_vecm[vecm_vars_full].copy()

print(f"✅ Variables del modelo completo: {vecm_vars_full}")
print(f"📊 Observaciones: {len(df_vecm_full)}")

# Test de cointegración con 5 variables
if coint_johansen is not None:
    johansen_result = coint_johansen(df_vecm_full.values, det_order=1, k_ar_diff=2)
    print(f"\n📊 TEST JOHANSEN (5 variables):")
    print(f"Estadístico traza: {johansen_result.lr1}")
    print(f"Valores críticos 5%: {johansen_result.cvt[:, 1]}")
    n_coint = sum(johansen_result.lr1 > johansen_result.cvt[:, 1])
    print(f"✅ Relaciones de cointegración: {n_coint}")

print(f"\n🎯 Usar modelo de 5 variables (siguiendo Talvi)")

🔍 PASO 4: COINTEGRACIÓN
✅ Variables del modelo completo: ['log_gdp_brazil', 'log_ip_g7', 'log_tot_brazil', 'us_10y', 'risk_spread']
📊 Observaciones: 99

📊 TEST JOHANSEN (5 variables):
Estadístico traza: [83.60630127 51.50845762 24.14151017 11.13208275  1.10135229]
Valores críticos 5%: [79.3422 55.2459 35.0116 18.3985  3.8415]
✅ Relaciones de cointegración: 1

🎯 Usar modelo de 5 variables (siguiendo Talvi)


In [12]:
# PASO 5: ESTIMACIÓN VECM COMPLETO (5 VARIABLES)
print("="*50)
print("📊 PASO 5: VECM COMPLETO - BRASIL")
print("="*50)

# Estimar VECM con 5 variables y 1 relación de cointegración
try:
    vecm_model = VECM(df_vecm_full, k_ar_diff=2, coint_rank=1, deterministic='ci')
    vecm_fit = vecm_model.fit()
    
    print("✅ VECM completo estimado exitosamente")
    print(f"📊 Observaciones: {vecm_fit.nobs}")
    
    # Extraer coeficientes
    beta = vecm_fit.beta.flatten() if vecm_fit.beta.ndim > 1 else vecm_fit.beta
    alpha = vecm_fit.alpha.flatten() if vecm_fit.alpha.ndim > 1 else vecm_fit.alpha
    
    print(f"\n🔍 RELACIÓN DE LARGO PLAZO (β):")
    for i, var in enumerate(vecm_vars_full):
        print(f"{var:<20}: {beta[i]:8.4f}")
    
    print(f"\n🔍 VELOCIDADES DE AJUSTE (α):")
    for i, var in enumerate(vecm_vars_full):
        print(f"{var:<20}: {alpha[i]:8.4f}")
    
    # Indicadores de vulnerabilidad completos
    brazil_vulnerability = {
        'ip_sensitivity': -beta[1],
        'tot_sensitivity': -beta[2], 
        'rate_sensitivity': -beta[3],
        'risk_sensitivity': -beta[4],
        'adjustment_speed': abs(alpha[0]),
        'vulnerability_score': abs(beta[1]) + abs(beta[2]) + abs(beta[3]) + abs(beta[4])
    }
    
    print(f"\n🎯 INDICADORES DE VULNERABILIDAD BRASIL:")
    print(f"Sensibilidad IP_G7:     {brazil_vulnerability['ip_sensitivity']:8.4f}")
    print(f"Sensibilidad TOT:       {brazil_vulnerability['tot_sensitivity']:8.4f}")
    print(f"Sensibilidad Tasas:     {brazil_vulnerability['rate_sensitivity']:8.4f}")
    print(f"Sensibilidad Riesgo:    {brazil_vulnerability['risk_sensitivity']:8.4f}")
    print(f"Velocidad Ajuste:       {brazil_vulnerability['adjustment_speed']:8.4f}")
    print(f"Score Vulnerabilidad:   {brazil_vulnerability['vulnerability_score']:8.4f}")
    
except Exception as e:
    print(f"❌ Error: {e}")

📊 PASO 5: VECM COMPLETO - BRASIL
✅ VECM completo estimado exitosamente
📊 Observaciones: 96

🔍 RELACIÓN DE LARGO PLAZO (β):
log_gdp_brazil      :   1.0000
log_ip_g7           :  -2.7915
log_tot_brazil      : -20.8638
us_10y              :  -0.0619
risk_spread         :   0.0103

🔍 VELOCIDADES DE AJUSTE (α):
log_gdp_brazil      :  -0.0016
log_ip_g7           :   0.0404
log_tot_brazil      :   0.0102
us_10y              :   0.1533
risk_spread         :  -2.4242

🎯 INDICADORES DE VULNERABILIDAD BRASIL:
Sensibilidad IP_G7:       2.7915
Sensibilidad TOT:        20.8638
Sensibilidad Tasas:       0.0619
Sensibilidad Riesgo:     -0.0103
Velocidad Ajuste:         0.0016
Score Vulnerabilidad:    23.7275


In [13]:
# PASO 6: ANÁLISIS FINAL DE VULNERABILIDAD BRASIL
print("="*60)
print("🇧🇷 REPORTE FINAL - VULNERABILIDAD EXTERNA BRASIL")
print("="*60)

# Clasificación de vulnerabilidad
tot_sens = brazil_vulnerability['tot_sensitivity']
ip_sens = brazil_vulnerability['ip_sensitivity']
overall_score = brazil_vulnerability['vulnerability_score']

print(f"📊 INDICADORES CLAVE:")
print(f"{'Sensibilidad a TOT:':<25} {tot_sens:>8.2f} (Muy Alta)")
print(f"{'Sensibilidad a IP_G7:':<25} {ip_sens:>8.2f} (Alta)")
print(f"{'Sensibilidad a Tasas US:':<25} {brazil_vulnerability['rate_sensitivity']:>8.3f} (Baja)")
print(f"{'Sensibilidad a Riesgo:':<25} {brazil_vulnerability['risk_sensitivity']:>8.3f} (Muy Baja)")
print(f"{'Velocidad de Ajuste:':<25} {brazil_vulnerability['adjustment_speed']:>8.4f} (Muy Lenta)")
print(f"{'Score Total:':<25} {overall_score:>8.2f}")

# Clasificación de riesgo
if overall_score > 20:
    risk_level = "🔴 MUY ALTO"
elif overall_score > 15:
    risk_level = "🟠 ALTO"
elif overall_score > 10:
    risk_level = "🟡 MEDIO"
else:
    risk_level = "🟢 BAJO"

print(f"\n🚨 NIVEL DE VULNERABILIDAD: {risk_level}")

print(f"\n📈 INTERPRETACIÓN:")
print(f"• 1% ↑ Terms of Trade → {tot_sens:.1f}% ↑ PIB Brasil")
print(f"• 1% ↑ Crecimiento G7 → {ip_sens:.1f}% ↑ PIB Brasil")
print(f"• Ajuste a equilibrio: {1/brazil_vulnerability['adjustment_speed']:.0f} trimestres")

print(f"\n🎯 CONCLUSIÓN:")
print(f"Brasil muestra ALTA vulnerabilidad a factores externos,")
print(f"especialmente a términos de intercambio y crecimiento global.")

# Guardar resultados
brazil_results = {
    'country': 'Brasil',
    'vulnerability_level': risk_level,
    'tot_elasticity': tot_sens,
    'ip_elasticity': ip_sens,
    'overall_score': overall_score,
    'model_period': f"{df_vecm_full.index.min()}-{df_vecm_full.index.max()}",
    'observations': vecm_fit.nobs
}

print(f"\n✅ ANÁLISIS COMPLETADO PARA BRASIL")

🇧🇷 REPORTE FINAL - VULNERABILIDAD EXTERNA BRASIL
📊 INDICADORES CLAVE:
Sensibilidad a TOT:          20.86 (Muy Alta)
Sensibilidad a IP_G7:         2.79 (Alta)
Sensibilidad a Tasas US:     0.062 (Baja)
Sensibilidad a Riesgo:      -0.010 (Muy Baja)
Velocidad de Ajuste:        0.0016 (Muy Lenta)
Score Total:                 23.73

🚨 NIVEL DE VULNERABILIDAD: 🔴 MUY ALTO

📈 INTERPRETACIÓN:
• 1% ↑ Terms of Trade → 20.9% ↑ PIB Brasil
• 1% ↑ Crecimiento G7 → 2.8% ↑ PIB Brasil
• Ajuste a equilibrio: 624 trimestres

🎯 CONCLUSIÓN:
Brasil muestra ALTA vulnerabilidad a factores externos,
especialmente a términos de intercambio y crecimiento global.

✅ ANÁLISIS COMPLETADO PARA BRASIL
